# AdaBoost
---

__This Notebook__

- try deeper trees

__Results__ 

- deeper trees easily overfit so, it's possible 97% sensitivity is the best, or close to the best we can get with AdaBoost
- `min_samples_split=5` also started overfitting


## Setup

In [1]:
import re
import os
import time
import joblib 

import numpy as np
import pandas as pd
import scipy.sparse as sp
import matplotlib.pyplot as plt

from datetime import datetime
from sklearn.metrics import confusion_matrix

dt_object = datetime.fromtimestamp(time.time())
day, T = str(dt_object).split('.')[0].split(' ')
print('Revised on: ' + day)

Revised on: 2021-02-07


## Load

In [2]:
# load target vector
raw_path = os.path.join("data","1_raw")
y_df = pd.read_csv(os.path.join(raw_path, 'y_train.csv'))
y_array = np.array(y_df.iloc[:,0].ravel())

y = y_array.copy()

# transform y_array into int type
y[y=='ham'] = 0
y[y=='spam'] = 1
y = y.astype('int')

# load matrix
proc_dir = os.path.join("data", "2_processed")
X_tfidf_svd800_spamcos = sp.load_npz(os.path.join(proc_dir, 'X_tfidf_svd800_spamcos.npz'))

## Helpful Functions


In [3]:
# train classifier and time it
def time_deco(func):
    def wrapper(clf):
        start = time.time()
        func(clf)
        m,s = divmod(time.time() - start, 60)
        print(f'Elapsed: {m:0.0f}m {s:0.0f}s')
    return wrapper

@time_deco
def fit_clf(clf):
    clf.fit(X_train, y_train)
    
# evaluate classifier
def eval_clf(y_val, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_val, 
                                      y_pred).ravel()
    confmat_df = pd.DataFrame(
        np.array(([tn, fp], [fn, tp])),
        columns=['pred_neg', 'pred_pos'], 
        index=['cond_neg', 'cond_pos']
    )
    # unpack metrics
    acc = (tp + tn) / (tp + tn + fp + fn)
    tpr = tp / (tp + fn)
    tnr = tn / (tn + fp)
    # print results
    print(confmat_df)
    print(f'acc: {acc:0.4f}')
    print(f'tpr: {tpr:0.4f}')
    print(f'tnr: {tnr:0.4f}')

def extract_df(gd):
    gd_res = gd.cv_results_
    df = pd.concat([
                    pd.DataFrame(gd_res["params"]),
                    pd.DataFrame(gd_res["mean_train_acc"], columns=["mean_train_acc"]),
                    pd.DataFrame(gd_res["mean_train_tpr"], columns=["mean_train_tpr"]),
                    pd.DataFrame(gd_res["mean_train_tnr"], columns=["mean_train_tnr"]),
                    pd.DataFrame(gd_res["mean_test_acc"], columns=["mean_val_acc"]),
                    pd.DataFrame(gd_res["mean_test_tpr"], columns=["mean_val_tpr"]),
                    pd.DataFrame(gd_res["mean_test_tnr"], columns=["mean_val_tnr"]),
                    pd.DataFrame(gd_res["mean_fit_time"], columns=["mean_fit_time"]),
                    pd.DataFrame(gd_res["std_test_acc"], columns=["std_val_acc"]),
                    pd.DataFrame(gd_res["std_test_tpr"], columns=["std_val_tpr"]),
                    pd.DataFrame(gd_res["std_test_tnr"], columns=["std_val_tnr"]),
                   ]
                   , axis=1)
    return df

### Train Test Split

##  AdaBoost (Adaptive Boosting) 


The first successful boosting algorithm for binary classification. Many current boosting methods build on it.

Scikit-Learn implements [SAMME](https://web.stanford.edu/~hastie/Papers/samme.pdf): *Stagewise Additive Modeling using a Multiclass Esponential loss function.* With two classes, SAMME works just as AdaBoost would. If predictions can estimate class probabilities, Scikit-Learn uses a variant of SAMME called *SAMME.R* (R=Real).

```
class AdaBoostClassifier(ClassifierMixin, BaseWeightBoosting):
    """An AdaBoost classifier.
    An AdaBoost [1] classifier is a meta-estimator that begins by fitting a
    classifier on the original dataset and then fits additional copies of the
    classifier on the same dataset but where the weights of incorrectly
    classified instances are adjusted such that subsequent classifiers focus
    more on difficult cases.
    
    This class implements the algorithm known as AdaBoost-SAMME [2].

```

See [docs](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html), [code](https://github.com/scikit-learn/scikit-learn/blob/95119c13a/sklearn/ensemble/_weight_boosting.py#L285)


__Some defaults:__

- `base_estimator`: the default (None) is a DecisionTreeClassifier initialized with max_depth=1
- `n_estimators`: default=50
- `learning_rate`: default=1, shrinks contribution of each classifier; there's a trade-off with n_estimators
- `algorithm{‘SAMME’, ‘SAMME.R’}`: default=’SAMME.R’
- `random_state`: default=None

__Some methods:__

- `.get_params`: returns the parameters
- `.decision_function`: evaluates the decision function for the samples in X, if binary from -1 to 1
- `.predict`: performs the classification, equivalent to decision_function with a 0 threshold
- `.predict_proba`: predict class probabilities for X. 


## GridSearchCV

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, recall_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

def gridsearch_wrapper(X, y, param_grid, k=5, n_jobs=6, max_depth=1, min_samples_split=5):
    """
    Performs a grid search with
    Args:
        X: numeric matrix
        y: target variable
        k: number of CV folds
        n_jobs: number of logical cores
    """
    start_time = time.time()

    # split into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X, 
                                                      y, 
                                                      stratify=y,
                                                      random_state=42)

    # setup scorers
    scorers = {
        'acc': make_scorer(accuracy_score),
        'tpr': make_scorer(recall_score, pos_label=1), # sensitivity, recall
        'tnr': make_scorer(recall_score, pos_label=0) # specificity, selectivity
    }

    # instantiate estimator
    clf =  AdaBoostClassifier(
        base_estimator=DecisionTreeClassifier(
            max_depth=max_depth,
            min_samples_split=min_samples_split),
        random_state=42
    )

    # instantiate k-fold gridsearch
    cv_folds = StratifiedKFold(n_splits=k)
    
    grid_search_clf = GridSearchCV(clf, 
                                   param_grid,
                                   scoring=scorers, 
                                   refit='tpr', 
                                   cv=cv_folds, 
                                   return_train_score=True, 
                                   n_jobs=n_jobs,
                                   verbose=1)
    
    # train models
    grid_search_clf.fit(X_train, y_train)

    # predict
    y_pred = grid_search_clf.predict(X_val)
    print(f'Best params: {grid_search_clf.best_params_}')

    # eval metrics
    print('Evaluation metrics:')
    eval_clf(y_val, y_pred)
    
    return grid_search_clf

In [6]:
params = {
    'n_estimators': [10, 50, 100],
    'learning_rate': [.001, .01]
}

In [7]:
gridsearch_clf1 = gridsearch_wrapper(X_tfidf_svd800_spamcos,
                                     y, 
                                     params,
                                     k=10, 
                                     max_depth=1, 
                                     n_jobs=-1)

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  6.0min finished


Best params: {'learning_rate': 0.001, 'n_estimators': 10}
Evaluation metrics:
          pred_neg  pred_pos
cond_neg       839         7
cond_pos         2       127
acc: 0.9908
tpr: 0.9845
tnr: 0.9917


In [15]:
df = extract_df(gridsearch_clf1)
df.sort_values(by=['mean_val_tpr'], ascending=False)

,learning_rate,n_estimators,mean_train_acc,mean_train_tpr,mean_train_tnr,mean_val_acc,mean_val_tpr,mean_val_tnr,mean_fit_time,std_val_acc,std_val_tpr,std_val_tnr
0,0.001,10,0.994644,0.975659,0.997547,0.993157,0.969096,0.996849,4.802966,0.004055,0.025236,0.003858
1,0.001,50,0.994644,0.975659,0.997547,0.993157,0.969096,0.996849,37.895976,0.004055,0.025236,0.003858
2,0.001,100,0.994644,0.975659,0.997547,0.993157,0.969096,0.996849,74.574805,0.004055,0.025236,0.003858
3,0.010,10,0.994644,0.975659,0.997547,0.993157,0.969096,0.996849,8.171846,0.004055,0.025236,0.003858
4,0.010,50,0.994644,0.975659,0.997547,0.993157,0.969096,0.996849,39.862671,0.004055,0.025236,0.003858
5,0.010,100,0.994644,0.975659,0.997547,0.993157,0.969096,0.996849,76.285308,0.004055,0.025236,0.003858


### Deeper trees

In [18]:
gridsearch_clf2 = gridsearch_wrapper(X_tfidf_svd800_spamcos, 
                                     y, 
                                     params,
                                     k=10, 
                                     max_depth=5,
                                     n_jobs=-1)

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  7.2min finished


Best params: {'learning_rate': 0.01, 'n_estimators': 10}
Evaluation metrics:
          pred_neg  pred_pos
cond_neg       839         7
cond_pos         3       126
acc: 0.9897
tpr: 0.9767
tnr: 0.9917


In [20]:
df = extract_df(gridsearch_clf2)
df.sort_values(by=['mean_val_tpr'], ascending=False)

,learning_rate,n_estimators,mean_train_acc,mean_train_tpr,mean_train_tnr,mean_val_acc,mean_val_tpr,mean_val_tnr,mean_fit_time,std_val_acc,std_val_tpr,std_val_tnr
3,0.010,10,1.000000,1.000000,1.0,0.989399,0.969096,0.992510,9.844403,0.005815,0.025236,0.005699
4,0.010,50,1.000000,1.000000,1.0,0.989399,0.969096,0.992510,29.583053,0.005815,0.025236,0.005699
5,0.010,100,1.000000,1.000000,1.0,0.989399,0.969096,0.992510,40.069161,0.005815,0.025236,0.005699
0,0.001,10,0.999506,0.996275,1.0,0.988716,0.963968,0.992512,17.247713,0.005731,0.026231,0.006700
1,0.001,50,0.999658,0.997421,1.0,0.988716,0.963968,0.992512,54.347530,0.005731,0.026231,0.006700
2,0.001,100,0.999658,0.997421,1.0,0.988716,0.963968,0.992512,81.525042,0.005731,0.026231,0.006700


In [27]:
gridsearch_clf3 = gridsearch_wrapper(X_tfidf_svd800_spamcos, 
                                     y, 
                                     {'n_estimators': [5, 10, 20],
                                      'learning_rate': [.001, .01, .1, 1]},
                                     k=10, 
                                     max_depth=2,
                                     n_jobs=-1)

Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   38.4s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  2.8min finished


Best params: {'learning_rate': 0.1, 'n_estimators': 20}
Evaluation metrics:
          pred_neg  pred_pos
cond_neg       843         3
cond_pos         2       127
acc: 0.9949
tpr: 0.9845
tnr: 0.9965


In [28]:
df = extract_df(gridsearch_clf3)
df.sort_values(by=['mean_val_tpr'], ascending=False)

,learning_rate,n_estimators,mean_train_acc,mean_train_tpr,mean_train_tnr,mean_val_acc,mean_val_tpr,mean_val_tnr,mean_fit_time,std_val_acc,std_val_tpr,std_val_tnr
8,0.100,20,0.998974,0.992269,1.000000,0.994183,0.969163,0.998030,17.887918,0.003445,0.022339,0.003631
1,0.001,10,0.995783,0.976518,0.998730,0.992133,0.966532,0.996061,7.444222,0.004342,0.030529,0.005282
2,0.001,20,0.995783,0.976518,0.998730,0.992133,0.966532,0.996061,16.940128,0.004342,0.030529,0.005282
3,0.010,5,0.995859,0.975945,0.998905,0.992815,0.966532,0.996849,4.642983,0.003892,0.030529,0.003858
4,0.010,10,0.996125,0.976518,0.999124,0.992474,0.966532,0.996455,9.401912,0.003994,0.030529,0.004472
5,0.010,20,0.996201,0.976232,0.999255,0.992815,0.966532,0.996849,18.485027,0.003892,0.030529,0.003858
6,0.100,5,0.996125,0.975086,0.999343,0.992815,0.966532,0.996849,5.553625,0.003892,0.030529,0.003858
7,0.100,10,0.996733,0.977949,0.999606,0.993498,0.966532,0.997636,10.071650,0.004453,0.030529,0.003609
0,0.001,5,0.995707,0.975660,0.998774,0.992134,0.963968,0.996455,3.865002,0.004062,0.030839,0.004806
10,1.000,10,1.000000,1.000000,1.000000,0.992473,0.963900,0.996846,8.946135,0.004280,0.031044,0.002364


In [30]:
# persist
save_path = os.path.join("data", "3_modeling", "02072021_ada_gridsearch3.joblib")
joblib.dump(gridsearch_clf3, save_path)

['data\\3_modeling\\02072021_ada_gridsearch3.joblib']

In [34]:
gridsearch_clf4 = gridsearch_wrapper(X_tfidf_svd800_spamcos, 
                                     y, 
                                     {'n_estimators': [10, 20],
                                      'learning_rate': [.001, .01, .1]},
                                     k=10, 
                                     min_samples_split=5,
                                     n_jobs=-1)

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   33.6s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   58.0s finished


Best params: {'learning_rate': 0.001, 'n_estimators': 10}
Evaluation metrics:
          pred_neg  pred_pos
cond_neg       839         7
cond_pos         2       127
acc: 0.9908
tpr: 0.9845
tnr: 0.9917


In [35]:
df = extract_df(gridsearch_clf3)
df.sort_values(by=['mean_val_tpr'], ascending=False)

,learning_rate,n_estimators,mean_train_acc,mean_train_tpr,mean_train_tnr,mean_val_acc,mean_val_tpr,mean_val_tnr,mean_fit_time,std_val_acc,std_val_tpr,std_val_tnr
8,0.100,20,0.998974,0.992269,1.000000,0.994183,0.969163,0.998030,17.887918,0.003445,0.022339,0.003631
1,0.001,10,0.995783,0.976518,0.998730,0.992133,0.966532,0.996061,7.444222,0.004342,0.030529,0.005282
2,0.001,20,0.995783,0.976518,0.998730,0.992133,0.966532,0.996061,16.940128,0.004342,0.030529,0.005282
3,0.010,5,0.995859,0.975945,0.998905,0.992815,0.966532,0.996849,4.642983,0.003892,0.030529,0.003858
4,0.010,10,0.996125,0.976518,0.999124,0.992474,0.966532,0.996455,9.401912,0.003994,0.030529,0.004472
5,0.010,20,0.996201,0.976232,0.999255,0.992815,0.966532,0.996849,18.485027,0.003892,0.030529,0.003858
6,0.100,5,0.996125,0.975086,0.999343,0.992815,0.966532,0.996849,5.553625,0.003892,0.030529,0.003858
7,0.100,10,0.996733,0.977949,0.999606,0.993498,0.966532,0.997636,10.071650,0.004453,0.030529,0.003609
0,0.001,5,0.995707,0.975660,0.998774,0.992134,0.963968,0.996455,3.865002,0.004062,0.030839,0.004806
10,1.000,10,1.000000,1.000000,1.000000,0.992473,0.963900,0.996846,8.946135,0.004280,0.031044,0.002364


---